-----------

# TRABAJO PRACTICO FINAL

### DIGITAL HOUSE: Data Science
### Junio 2022

<br>

## GRUPO 1
- Integrantes:
    - Gonzalo Barbot - PM: Project Manager
    - Agustin Stigliano - CDM: Code Development Manager
    - Fernando Dupont - PDM: Presentation Development Manager
    
<br>

## TEMA: Pronósticos sobre Series Temporales
<br>

-----

# NOTEBOOK MACHINE LEARNING:


In [1]:
import SmartPandas, SmartSeries
from SmartPandas import x_en
import APP_REG_LOG_EST as reg
import APP_ARIMA as arima
import APP_XGBOOST as xgb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import pickle

import logging
import datetime as dt
formatter = '%(levelname)s:\n %(message)s | %(asctime)s\n| line %(lineno)d\n'
logging.basicConfig(filename='logfile_Machine_learning.log', level=logging.INFO, force=True, filemode='w', **{'format':formatter})
logger = logging.getLogger()

Libreria SmartPandas:
	Clases:
		- PandasTransform, PandasFeatureUnion
Libreria SmartSeries:
	Clases:
		- VectorBuilder() 
	Funciones:
		- load_dataset
>> APP REG_LOG_EST: APLICACION PARA ENTRENAR Y EXPORTAR UN MODELO DE REGRESION LINEAL CON TRANSFORMACION LOGARITMICA APLICANDO ESTACIONALIDAD
>> APP ARIMA: Recibe una serie, y aplica logaritmo, 3 procesos de diferenciacion y un proceso de optimización, luego grafica para visualizar resultados y seleccionar parámetros ARIMA
>> APP XGBOST: APLICACION XGBOOST PARA TIME SERIES FORECASTING

xgboost version actual: 1.5.0
Version de diseño en 14/6/2022
	xgboost version: 1.6.0


In [6]:
logger.info('{}) STARTING PROGRAM'.format(x_en('log_ML')))
df = SmartSeries.load_dataset('df_targets.csv', freq='D', dropnan=False)


display (df)
print (df.index)
display (SmartPandas.data_info(df))


Serie de tiempo: date



,Natural Products,Bakery,Meat
date,,,
2012-04-10,101.781519,161.637857,353.698462
2012-04-11,101.030875,123.934848,222.319762
2012-04-12,114.137453,158.606944,521.658136
2012-04-13,106.682081,182.599318,521.257895
2012-04-14,126.068018,125.618571,327.602143
...,...,...,...
2013-06-26,103.016559,149.655882,223.138889
2013-06-27,100.582025,151.791714,392.826829
2013-06-28,87.684000,116.581364,228.294590


DatetimeIndex(['2012-04-10', '2012-04-11', '2012-04-12', '2012-04-13',
               '2012-04-14', '2012-04-15', '2012-04-16', '2012-04-17',
               '2012-04-18', '2012-04-19',
               ...
               '2013-06-21', '2013-06-22', '2013-06-23', '2013-06-24',
               '2013-06-25', '2013-06-26', '2013-06-27', '2013-06-28',
               '2013-06-29', '2013-06-30'],
              dtype='datetime64[ns]', name='date', length=447, freq='D')


info de data,columna,Nan,pct_nan,dtype,count,pct_reg,count_unique
0,Natural Products,1,0.22,float64,446,99.78,446
1,Bakery,1,0.22,float64,446,99.78,446
2,Meat,1,0.22,float64,446,99.78,446


In [8]:
df.isna()

,Natural Products,Bakery,Meat
date,,,
2012-04-10,False,False,False
2012-04-11,False,False,False
2012-04-12,False,False,False
2012-04-13,False,False,False
2012-04-14,False,False,False
...,...,...,...
2013-06-26,False,False,False
2013-06-27,False,False,False
2013-06-28,False,False,False


**MODELO DE REGRESION LINEAL CON TRANSFORMACIÓN LOGARÍTMICA Y ESTACIONALIDAD**

**ESTUDIO DE ESTACIONALIDAD SOBRE LOS RESIDUOS Y SOBRE PROCESO DE DIFERENCIACION ESTANDAR**

**MODELO ARIMA**

**MODELO XGBOOST**

**MODELO DE ENSAMBLE**

**CONCLUSIONES**